# Learning: Building a Custom Planner

This notebook guides you through building your own path planning algorithm. **You'll need to fill in the missing code!**

## What You'll Learn

1. Understanding the planner interface (`BasePlanner`)
2. Implementing a simple waypoint-based planner
3. Implementing a costmap-aware planner
4. Testing your implementation

## Instructions

- Read each section carefully
- Look for `# TODO:` comments - these indicate where you need to write code
- Fill in the `...` placeholders
- Run cells as you complete them to test your work

## Prerequisites

- Understanding of planners (see [Planner Tutorial](../tutorials/planner_tutorial.ipynb))
- Understanding of costmaps (see [Costmap Tutorial](../tutorials/costmap_tutorial.ipynb))
- Basic numpy knowledge

In [ ]:
import sys
sys.path.insert(0, '../../src')

import numpy as np
import matplotlib.pyplot as plt
from simple_autonomous_car import (
    Track,
    Car,
    CarState,
    BasePlanner,
    GridCostmap,
    GroundTruthMap,
    PerceivedMap,
    LiDARSensor,
)

## Step 1: Understanding the BasePlanner Interface

All planners must inherit from `BasePlanner` and implement the `plan()` method.

In [ ]:
from simple_autonomous_car.planning.base_planner import BasePlanner
import inspect

print("BasePlanner interface:")
print(f"  plan(car_state, perception_data, costmap, goal) -> np.ndarray")
print(f"  Returns: Array of shape (N, 2) with [x, y] waypoints")
print("\nKey points:")
print("  - Must return np.ndarray, even if empty (use np.array([]).reshape(0, 2))")
print("  - Can use perception_data, costmap, or goal (all optional)")
print("  - Should check if enabled before planning")

## Step 2: Build a Simple Waypoint Planner

Let's start with a simple planner that generates waypoints ahead of the car.

In [ ]:
class SimpleWaypointPlanner(BasePlanner):
    """
    A simple planner that generates waypoints in a straight line ahead of the car.
    
    TODO: Fill in the implementation!
    """
    
    def __init__(
        self,
        lookahead_distance: float = 20.0,
        waypoint_spacing: float = 2.0,
        name: str = "simple_waypoint_planner",
    ):
        """
        Initialize the planner.
        
        TODO:
        1. Call super().__init__(name=name)
        2. Store lookahead_distance and waypoint_spacing as instance variables
        """
        # TODO: Initialize parent class
        super().__init__(...)
        
        # TODO: Store parameters
        self.lookahead_distance = ...
        self.waypoint_spacing = ...
    
    def plan(
        self,
        car_state: CarState,
        perception_data: dict = None,
        costmap = None,
        goal = None,
    ) -> np.ndarray:
        """
        Generate a simple straight-line plan ahead of the car.
        
        TODO:
        1. Check if enabled, return empty array if not
        2. Get car position: car_state.position()
        3. Get car heading: car_state.heading
        4. Calculate number of waypoints: int(lookahead_distance / waypoint_spacing) + 1
        5. Generate waypoints:
           - For each waypoint i from 0 to num_waypoints:
             - distance = i * waypoint_spacing
             - x = car_pos[0] + distance * cos(heading)
             - y = car_pos[1] + distance * sin(heading)
             - Append [x, y] to waypoints list
        6. Return np.array(waypoints)
        """
        # TODO: Check if enabled
        if not ...:
            return np.array([]).reshape(0, 2)
        
        # TODO: Get car state
        car_pos = ...
        car_heading = ...
        
        # TODO: Calculate number of waypoints
        num_waypoints = ...
        
        # TODO: Generate waypoints
        waypoints = []
        for i in range(num_waypoints):
            distance = ...
            x = ...
            y = ...
            waypoints.append([x, y])
        
        return np.array(waypoints)

print("✓ Simple waypoint planner structure created!")
print("  Now fill in the TODOs!")

## Step 3: Test Your Simple Planner

In [ ]:
# Setup test
track = Track.create_simple_track(length=80.0, width=40.0, track_width=5.0)
start_point, start_heading = track.get_point_at_distance(0.0)
car_state = CarState(x=start_point[0], y=start_point[1], heading=start_heading, velocity=8.0)

# TODO: Create your planner
my_planner = SimpleWaypointPlanner(
    lookahead_distance=30.0,
    waypoint_spacing=3.0,
)

# TODO: Generate a plan
plan = my_planner.plan(car_state)

print(f"✓ Plan generated: {len(plan)} waypoints")
if len(plan) > 0:
    print(f"  First waypoint: ({plan[0][0]:.2f}, {plan[0][1]:.2f})")
    print(f"  Last waypoint: ({plan[-1][0]:.2f}, {plan[-1][1]:.2f})")

In [ ]:
# Visualize your plan
from simple_autonomous_car.visualization import plot_car

fig, ax = plt.subplots(figsize=(12, 10))

# Plot track
track.visualize(ax=ax, frame="global")

# TODO: Plot your plan
my_planner.visualize(ax=ax, car_state=car_state, plan=plan, frame="global", color="green", label="Your Plan", show_waypoints=True)

# Plot car
car = Car(initial_state=car_state)
plot_car(car, ax=ax, show_heading=True)

ax.set_title("Your Simple Waypoint Planner")
ax.legend()
plt.show()

print("✓ Visualization complete!")

## Step 4: Build a Costmap-Aware Planner

Now let's build a more advanced planner that avoids obstacles using a costmap!

In [ ]:
class CostmapAwarePlanner(BasePlanner):
    """
    A planner that uses costmap to avoid obstacles.
    
    TODO: Fill in the implementation!
    """
    
    def __init__(
        self,
        lookahead_distance: float = 30.0,
        waypoint_spacing: float = 2.0,
        max_cost_threshold: float = 0.5,
        name: str = "costmap_aware_planner",
    ):
        """
        Initialize the costmap-aware planner.
        
        TODO:
        1. Call super().__init__(name=name)
        2. Store lookahead_distance, waypoint_spacing, max_cost_threshold
        """
        # TODO: Initialize
        super().__init__(...)
        self.lookahead_distance = ...
        self.waypoint_spacing = ...
        self.max_cost_threshold = ...
    
    def plan(
        self,
        car_state: CarState,
        perception_data: dict = None,
        costmap = None,
        goal = None,
    ) -> np.ndarray:
        """
        Generate a plan that avoids high-cost areas.
        
        TODO:
        1. Check if enabled, return empty if not
        2. If no costmap, fall back to simple straight-line plan (like SimpleWaypointPlanner)
        3. Generate initial waypoints in straight line (same as SimpleWaypointPlanner)
        4. For each waypoint:
           - Get cost at waypoint using costmap.get_cost(waypoint, frame="global", car_state=car_state)
           - If cost > max_cost_threshold:
             - Try to find alternative position (e.g., offset perpendicular to path)
             - Or skip this waypoint
        5. Return filtered/adjusted waypoints
        
        HINT: Start simple - just filter out waypoints with high cost, then improve!
        """
        # TODO: Check if enabled
        if not ...:
            return np.array([]).reshape(0, 2)
        
        # TODO: Get car state
        car_pos = ...
        car_heading = ...
        
        # TODO: Generate initial waypoints (straight line)
        num_waypoints = int(self.lookahead_distance / self.waypoint_spacing) + 1
        waypoints = []
        for i in range(num_waypoints):
            distance = i * self.waypoint_spacing
            x = car_pos[0] + distance * np.cos(car_heading)
            y = car_pos[1] + distance * np.sin(car_heading)
            waypoints.append([x, y])
        
        # TODO: Filter/adjust waypoints based on costmap
        if costmap is not None and costmap.enabled:
            safe_waypoints = []
            for waypoint in waypoints:
                cost = ...
                if cost <= ...:
                    safe_waypoints.append(waypoint)
            waypoints = safe_waypoints
        
        return np.array(waypoints)

print("✓ Costmap-aware planner structure created!")
print("  Now fill in the TODOs!")

## Step 5: Test Your Costmap-Aware Planner

In [ ]:
# Setup with costmap
track = Track.create_simple_track(length=80.0, width=40.0, track_width=5.0)
ground_truth_map = GroundTruthMap(track)
perceived_map = PerceivedMap(ground_truth_map)

start_point, start_heading = track.get_point_at_distance(0.0)
car = Car(initial_state=CarState(x=start_point[0], y=start_point[1], heading=start_heading, velocity=8.0))

lidar = LiDARSensor(ground_truth_map, perceived_map, max_range=30.0, name="lidar")
car.add_sensor(lidar)

# Create costmap
costmap = GridCostmap(width=60.0, height=60.0, resolution=1.0, frame="ego")

# Get perception and update costmap
perception_data = car.sense_all(environment_data={"ground_truth_map": ground_truth_map})
costmap.update(perception_data, car.state)

# TODO: Create your costmap-aware planner
my_planner = CostmapAwarePlanner(
    lookahead_distance=30.0,
    waypoint_spacing=2.0,
    max_cost_threshold=0.5,
)

# TODO: Generate plan with costmap
plan = my_planner.plan(car.state, perception_data=perception_data, costmap=costmap)

print(f"✓ Plan generated: {len(plan)} waypoints")
print(f"  (Should be fewer than straight-line plan if obstacles detected)")

In [ ]:
# Visualize plan with costmap
from simple_autonomous_car.visualization import plot_car

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# World frame
ax = axes[0]
track.visualize(ax=ax, frame="global")
costmap.visualize(ax=ax, car_state=car.state, frame="global", alpha=0.3)
my_planner.visualize(ax=ax, car_state=car.state, plan=plan, frame="global", color="green", label="Your Plan", show_waypoints=True)
plot_car(car, ax=ax, show_heading=True)
ax.set_title("Costmap-Aware Plan - World Frame")
ax.legend()

# Ego frame
ax = axes[1]
costmap.visualize(ax=ax, car_state=car.state, frame="ego", alpha=0.3)
plan_ego = np.array([car.state.transform_to_car_frame(p) for p in plan])
my_planner.visualize(ax=ax, car_state=car.state, plan=plan_ego, frame="ego", color="green", label="Your Plan", show_waypoints=True)
ax.set_title("Costmap-Aware Plan - Ego Frame")
ax.legend()

plt.tight_layout()
plt.show()

print("✓ Visualization complete!")
print("  Your plan should avoid high-cost (red) areas!")

## Summary

Congratulations! You've built two different planners! 🎉

### What You Learned

1. ✅ **Planner interface**: Understanding `BasePlanner` and the `plan()` method
2. ✅ **Simple planning**: Generating waypoints in a straight line
3. ✅ **Costmap integration**: Using costmaps to avoid obstacles
4. ✅ **Waypoint filtering**: Adjusting plans based on cost values

### Key Concepts

- **Lookahead distance**: How far ahead to plan
- **Waypoint spacing**: Distance between waypoints
- **Cost threshold**: Maximum acceptable cost for waypoints
- **Frame handling**: Plans are typically in global frame

### Next Steps

- Improve obstacle avoidance (try offsetting waypoints around obstacles)
- Add path smoothing
- Implement A* or RRT (see [Advanced Planning](../learning/advanced_planning_algorithms.ipynb))
- Add dynamic replanning
- Consider vehicle dynamics in planning